# **1. 서울 열린데이터 광장**
<a href="https://data.seoul.go.kr/">서울 열린데이터 광장</a>(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.

```
서울시 공공자전거 실시간 대여정보
http://openapi.seoul.go.kr:8088/(인증키)/json/bikeList/1/5/
4a416c59516f686a3130357748786e69
```

# **2. 서울시 공공자전거 실시간 대여정보**

In [1]:
import requests
import pandas as pd
import folium

In [2]:
base_url = 'http://openapi.seoul.go.kr:8088/4a416c59516f686a3130357748786e69/json/bikeList/1/3/'
response = requests.get(base_url)
response # 인증에 성공했다면 Response 200

<Response [200]>

In [3]:
json_data = response.json()
json_data

{'rentBikeStatus': {'list_total_count': 3,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '26',
    'shared': '173',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '25',
    'shared': '179',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'}]}}

In [4]:
json_data.get('rentBikeStatus', {})

{'list_total_count': 3,
 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
 'row': [{'rackTotCnt': '15',
   'stationName': '102. 망원역 1번출구 앞',
   'parkingBikeTotCnt': '26',
   'shared': '173',
   'stationLatitude': '37.55564880',
   'stationLongitude': '126.91062927',
   'stationId': 'ST-4'},
  {'rackTotCnt': '14',
   'stationName': '103. 망원역 2번출구 앞',
   'parkingBikeTotCnt': '25',
   'shared': '179',
   'stationLatitude': '37.55495071',
   'stationLongitude': '126.91083527',
   'stationId': 'ST-5'},
  {'rackTotCnt': '13',
   'stationName': '104. 합정역 1번출구 앞',
   'parkingBikeTotCnt': '0',
   'shared': '0',
   'stationLatitude': '37.55073929',
   'stationLongitude': '126.91508484',
   'stationId': 'ST-6'}]}

In [5]:
json_data.get('rentBikeStatus', {}).get('RESULT', {})

{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'}

In [6]:
json_data.get('rentBikeStatus', {}).get('RESULT', {}).get('CODE', {})

'INFO-000'

In [7]:
json_data['rentBikeStatus']['RESULT']['CODE'] # 이렇게도 불러올 수 있지만 못가져올 경우를 생각해서 .get() 메서드로 하나하나씩 접근해서 가져옴

'INFO-000'

In [8]:
def fetch_bike_data():
    base_url = "http://openapi.seoul.go.kr:8088/4a416c59516f686a3130357748786e69/json/bikeList/"
    start = 1
    end = 1000
    step = 1000
    data_frames = []

    while True:
        url = f"{base_url}{start}/{end}/"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Status Code: {response.status_code}")
            break

        json_data = response.json()

        try:
            rent_bike_status = json_data["rentBikeStatus"]
            result_code = rent_bike_status["RESULT"]["CODE"]
        except KeyError:
            print("JSON 오류")
            break

        if result_code == "INFO-200":
            print("데이터 없음")
            break
        elif result_code == "INFO-000":
            print(f"시작: {start} 끝: {end}.")
            try:
                bike_data = rent_bike_status["row"]
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print("데이터를 찾을 수 없음")
        else:
            print(f"result code: {result_code}")
            break

        start += step
        end += step

    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [9]:
bike_data_df = fetch_bike_data()

시작: 1 끝: 1000.
시작: 1001 끝: 2000.
시작: 2001 끝: 3000.
JSON 오류


In [10]:
bike_data_df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,26,173,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,25,179,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,0,0,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,1,20,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,2,17,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2738,9,6180.양천향교역 3번출구,2,22,37.56978226,126.84175110,ST-3365
2739,10,6182.오스템임플란트비전타워,6,60,37.57143402,126.82884979,ST-3370
2740,10,6183.K스퀘어 마곡,7,70,37.56269455,126.82707977,ST-3371
2741,32,6184. 한강버스 마곡 선착장,47,147,37.57402039,126.84357452,ST-2492


In [11]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2743 entries, 0 to 2742
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2743 non-null   object
 1   stationName        2743 non-null   object
 2   parkingBikeTotCnt  2743 non-null   object
 3   shared             2743 non-null   object
 4   stationLatitude    2743 non-null   object
 5   stationLongitude   2743 non-null   object
 6   stationId          2743 non-null   object
dtypes: object(7)
memory usage: 150.1+ KB


In [12]:
'''
rackTotCnt	거치대개수
parkingBikeTotCnt	자전거주차총건수
shared	거치율
stationLatitude	위도
stationLongitude	경도
stationId	대여소ID
stationName	대여소이름
'''
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [13]:
bike_data_df.shape

(2743, 7)

In [14]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [15]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                            zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map